In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image directories
train_dir = '/kaggle/input/human-activity-image-data/har_structured/train'
test_dir = '/kaggle/input/human-activity-image-data/har_structured/test'

# Define ImageDataGenerators for training and testing datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size and image size
batch_size = 32
image_size = (224, 224)  # Target image size

# Flow images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size=batch_size,
    class_mode='categorical'  # for multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = image_size,
    batch_size=batch_size,
    class_mode='categorical'  # for multi-class classification
)

In [ ]:
import os
import matplotlib.pyplot as plt

# Get a batch of data from the generator
images, labels = next(train_generator)

print(labels.shape)
# Get the class names from the directory names
class_names = list(train_generator.class_indices.keys())
print(train_generator.class_indices)
# Visualize a sample image and its inferred label
sample_index = 0  # Index of the sample image to visualize
for i in range(0, 5):
    plt.imshow(images[i])
    plt.title("Label: " + class_names[labels[i].argmax()])
    plt.axis("off")
    plt.show()

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model

def inception_module(x, filters):
    """Inception module implementation."""
    # 1x1 conv branch
    branch1x1 = Conv2D(filters=filters[0], kernel_size=(1, 1), padding='same', activation='relu')(x)

    # 3x3 conv branch
    branch3x3 = Conv2D(filters=filters[1], kernel_size=(1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv2D(filters=filters[2], kernel_size=(3, 3), padding='same', activation='relu')(branch3x3)

    # 5x5 conv branch
    branch5x5 = Conv2D(filters=filters[3], kernel_size=(1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv2D(filters=filters[4], kernel_size=(5, 5), padding='same', activation='relu')(branch5x5)

    # Max pooling branch
    branch_pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = Conv2D(filters=filters[5], kernel_size=(1, 1), padding='same', activation='relu')(branch_pool)

    # Concatenate the outputs of all branches
    outputs = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return outputs

# Define GoogLeNet model
def googlenet(input_shape, num_classes):
    """GoogLeNet model implementation."""
    inputs = Input(shape=input_shape)

    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(filters=192, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, filters=[192, 96, 208, 16, 48, 64])
    x = inception_module(x, filters=[160, 112, 224, 24, 64, 64])
    x = inception_module(x, filters=[128, 128, 256, 24, 64, 64])
    x = inception_module(x, filters=[112, 144, 288, 32, 64, 64])
    x = inception_module(x, filters=[256, 160, 320, 32, 128, 128])
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, filters=[256, 160, 320, 32, 128, 128])
    x = inception_module(x, filters=[384, 192, 384, 48, 128, 128])
    
    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Create GoogLeNet model
input_shape = (224, 224, 3)  # Example input shape
num_classes = 15  # Number of output classes (ImageNet)
model = googlenet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

plot_model(model, to_file='/kaggle/working/model_plot.png', 
                  show_shapes=True, show_layer_names=True)

In [ ]:
from tensorflow.keras.optimizers import Adam
momentum_value = 0.9
optimizer = Adam(learning_rate=0.001, beta_1=momentum_value)
model.compile(optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
filepath = '/kaggle/working/model.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
                        verbose=1, save_best_only=True, mode='max')

In [ ]:
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=4, restore_best_weights=True)
callbacks_list = [checkpoint, early]

In [ ]:
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=test_generator,
    callbacks=callbacks_list
)

In [ ]:
# Training plots
model.save('/kaggle/working/model_final.keras')
epochs = [i for i in range(1, len(history.history['loss'])+1)]

plt.plot(epochs, history.history['loss'], color='blue', label="training_loss")
plt.plot(epochs, history.history['val_loss'], color='red', label="validation_loss")
plt.legend(loc='best')
plt.title('training')
plt.xlabel('epoch')
# plt.savefig(TRAINING_PLOT_FILE, bbox_inches='tight')
plt.show()

plt.plot(epochs, history.history['accuracy'], color='blue', label="training_accuracy")
plt.plot(epochs, history.history['val_accuracy'], color='red',label="validation_accuracy")
plt.legend(loc='best')
plt.title('validation')
plt.xlabel('epoch')
# plt.savefig(VALIDATION_PLOT_FILE, bbox_inches='tight')
plt.show()

In [ ]:
#make sure to load the best model
from tensorflow.keras.models import load_model
model = load_model('/kaggle/working/model_final.keras')

predictions = model.predict(test_generator, verbose=1)